In [ ]:

from IPython.display import clear_output

from optibook.synchronous_client import InfoOnly
import random
import time
import datetime

import logging
logger = logging.getLogger('client')
logger.setLevel('ERROR')


def book_pretty(price_book):
    if price_book is None:
        return 'NO BOOK'
    
    res = ''
    for level in reversed(range(6)):
        if level >= len(price_book.asks):
            res += "\n"
        else:
            ask = price_book.asks[level]
            res += f"        |  {ask.price:8.2f}  |  {ask.volume:5} \n"

    res += '------------------------------\n'

    for level in range(6):
        if level >= len(price_book.bids):
            res += "\n"
        else:
            bid = price_book.bids[level]
            res += f" {bid.volume:5}  |  {bid.price:8.2f}  |       \n"   

    return res

def tradeticks_pretty(disp_tradeticks):
    res = ''
    for t, dtt in reversed(disp_tradeticks):
        res += f"{t.strftime('%H:%M:%S')} -- {dtt.volume:5} @ {dtt.price:8.2f}\n"
    return res

i = InfoOnly()
#i = InfoOnly(host='host-to-connect-to')
i.connect()

In [ ]:
disp_tradeticks = []
disp_tradeticks_max_age = datetime.timedelta(seconds=20)
feedcode = 'PHILIPS_A'

while True:
    now = datetime.datetime.now()

    book = i.get_last_price_book(feedcode)
    trade_ticks = i.poll_new_trade_ticks(feedcode)

    for tt in trade_ticks:
        disp_tradeticks.append((now, tt))

    disp_tradeticks = [(t, dtt) for t, dtt in disp_tradeticks if now - t < disp_tradeticks_max_age]

    clear_output(wait=True)
    print(f"Order Book for '{feedcode}':")
    print(book_pretty(book))
    
    print(f"\nTrade ticks in last {disp_tradeticks_max_age.total_seconds()}s for '{feedcode}':")
    print(tradeticks_pretty(disp_tradeticks))
            
    time.sleep(0.020)